In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tifffile
from skimage import data
from matplotlib import pyplot as plt

def to_uint8(arr):
    return (arr * 255).astype('uint8')

In [ ]:
cells, nuclei = data.cells3d().max(axis=0)
cells = to_uint8(cells / cells.max())
nuclei = to_uint8(nuclei / nuclei.max())
fig, (a, b, c) = plt.subplots(1, 3)
a.imshow(cells, cmap='gray')
b.imshow(nuclei, cmap='gray')

import mergechannels as mc
c.imshow(mc.merge([cells, nuclei], ['Orange Hot', 'Cyan Hot']))

In [ ]:
skin = data.skin()
ch1, ch2, ch3 = skin[:, :, 0], skin[:, :, 1], skin[:, :, 2]
ch1 = to_uint8(ch1 / ch1.max())
ch2 = to_uint8(ch2 / ch2.max())
ch3 = to_uint8(ch3 / ch3.max())

fig, ((a,b), (c,d)) = plt.subplots(2,2)
# a.imshow(mc.apply_color_map(ch1, 'OPF orange'))
# b.imshow(mc.apply_color_map(ch2, 'OPF purple'))
# c.imshow(mc.apply_color_map(ch3, 'OPF fresh'))
a.imshow(ch1, cmap='gray')
b.imshow(ch2, cmap='gray')
c.imshow(ch3, cmap='gray')

import mergechannels as mc
d.imshow(mc.merge([ch1, ch2, ch3], ['OPF orange', 'OPF purple', 'OPF fresh'], blending='mean'))

In [ ]:
neuron, _, ganglia, dapi, brightfield = tifffile.imread('assets/samples/Rat_Hippocampal_Neuron.tif')
fig, axes = plt.subplots(1, 5, dpi=200)
a, b, c, d, e = axes
for ax in axes:
    ax.axis('off')
a.imshow(neuron, cmap='gray')
b.imshow(ganglia, cmap='gray')
c.imshow(dapi, cmap='gray')
d.imshow(brightfield, cmap='gray')

neuron = to_uint8(neuron / neuron.max())
ganglia = to_uint8(ganglia / ganglia.max())
dapi = to_uint8(dapi / dapi.max())
brightfield = to_uint8(brightfield / brightfield.max())

import mergechannels as mc
e.imshow(
    mc.merge(
        [neuron, ganglia, dapi],
        ['OPF orange', 'OPF purple', 'OPF fresh'],
        blending='max',
    ),
)

In [ ]:
def normalize(img: np.ndarray, max: int | None = None, min: int | None = None) -> np.ndarray:
    if max is None:
        max = img.max()
    if min is None:
        min = img.min()
    # Handle division by zero case
    if max == min:
        return np.full_like(img, 0, dtype=np.uint8)
    # Clip input values to max/min bounds before normalizing
    img_clipped = np.clip(img, min, max)
    return np.clip((img_clipped - min) / (max - min) * 255, 0, 255).astype(np.uint8)

neuron, _, ganglia, dapi, brightfield = tifffile.imread('assets/samples/Rat_Hippocampal_Neuron.tif')
dapi = to_uint8(dapi / dapi.max())
ganglia = normalize(ganglia, min=ganglia.min(), max=0.3 * ganglia.max())
brightfield = to_uint8(brightfield / brightfield.max())

fig, axes = plt.subplots(2, 2, figsize=(10, 10), dpi=200)
(a, b), (c, d) = axes
for ax in axes.ravel():
    ax.axis('off')
a.imshow(ganglia, cmap='gray')
b.imshow(brightfield, cmap='gray')
c.imshow(dapi, cmap='gray')
import mergechannels as mc
d.imshow(
    normalize(
        mc.merge(
            [
                brightfield,
                normalize(ganglia, min=10, max=0.9 * ganglia.max()),
                normalize(dapi, min=25, max=0.9 * dapi.max()),
            ],
            [
                'Grays',
                'betterOrange',
                'betterBlue'
            ],
            blending='mean',
        ),
        min=15,
        max=125,
    ),
)
fig.tight_layout()

In [ ]:
ganglia.max()

In [ ]:
golgi, mitochondria, nuclei = to_uint8(np.moveaxis(tifffile.imread('assets/samples/hela-cells.tif'), -1, 0) / 4096)

fig, axes = plt.subplots(1, 4, dpi=200)
a, b, c, d = axes
for ax in axes:
    ax.axis('off')
a.imshow(golgi, cmap='gray')
b.imshow(mitochondria, cmap='gray')
c.imshow(nuclei, cmap='gray')

import mergechannels as mc
d.imshow(
    mc.merge(
        [golgi, mitochondria, nuclei],
        ['OPF fresh', 'OPF orange', 'OPF purple'],
        blending='max',
    )
)


In [ ]:
def normalize(img: np.ndarray, max: int | None = None, min: int | None = None) -> np.ndarray:
    if max is None:
        max = img.max()
    if min is None:
        min = img.min()
    return np.clip((img - min) / (max - min) * 255, 0, 255).astype(np.uint8)

golgi, mitochondria, nuclei = normalize(np.moveaxis(tifffile.imread('assets/samples/hela-cells.tif'), -1, 0), min=0, max=3000)

fig, axes = plt.subplots(1, 2, dpi=200)
a, b = axes
for ax in axes:
    ax.axis('off')
fig.tight_layout()

import mergechannels as mc
a.imshow(
    mc.merge(
        [golgi, mitochondria, nuclei],
        ['OPF fresh', 'OPF orange', 'OPF purple'],
        blending='max',
    )
)
b.imshow(
    mc.merge(
        [golgi, mitochondria, nuclei],
        ['I Forest', 'I Red', 'I Bordeaux'],
        blending='min',
    )
)
